# Contrastive Learning using SimCLR

Created by [Àlex Solé](https://www.linkedin.com/in/alex-sole-gomez/) for the [Postgraduate course in artificial intelligence with deep learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) in [UPC School](https://www.talent.upc.edu/ing/) (2025).


# Contrastive Learning with SimCLR on STL10 Dataset

In this lab, we will implement and explore Self-Supervised Learning through Contrastive Learning using the SimCLR (Simple Framework for Contrastive Learning of Visual Representations) approach.

## What is SimCLR?

[SimCLR](https://arxiv.org/abs/2002.05709) is a simple framework for contrastive learning of visual representations developed by Google Research. It learns representations by maximizing agreement between differently augmented views of the same data example while minimizing agreement between augmented views of different examples.

<p align="center"><br>
<img src="https://github.com/telecombcn-dl/labs-all/raw/main/labs/contrastive_learning/images/simclr_architecture.png?raw=true" class="center" title="SimCLR" width="400"/>
</p><br>

## Our Dataset: STL10

We'll be using the [STL10](https://cs.stanford.edu/~acoates/stl10/) dataset, which is specifically designed for unsupervised learning:
- 100,000 unlabeled images for training
- 5,000 labeled training images and 8,000 test images
- 10 classes (airplane, bird, car, cat, deer, dog, horse, monkey, ship, truck)
- 96x96 pixel resolution color images
- Designed for developing unsupervised feature learning, deep learning, and self-taught learning algorithms

The dataset has two main splits:
1. **Unlabeled split**: 100,000 images without labels for self-supervised pretraining
2. **Labeled split**: A smaller set of labeled images (5,000 training, 8,000 testing) for supervised fine-tuning and evaluation

<p align="center"><br>
<img src="https://github.com/telecombcn-dl/labs-all/raw/main/labs/contrastive_learning/images/STL10_dataset.png?raw=true" class="center" title="STL-10" width="400"/>
</p><br>

## Approach

We will:
1. Create data augmentation pipelines
2. Implement a ResNet-based encoder network
3. Build a projection head for the contrastive learning task
4. Train the model using the contrastive loss function (NT-Xent)
5. Evaluate the learned representations on downstream tasks

Let's explore how we can learn meaningful representations without relying on labeled data!

In [ ]:
import torch
import torchvision.models as models
import numpy as np
import os
import torch
import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import STL10
from torch.utils.data import DataLoader
from torch.multiprocessing import cpu_count
import torchvision.transforms as T

To ensure reproducibility of the experiments, we can set the seed to a fixed number.

In [ ]:
SEED = 77777
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
if not torch.cuda.is_available():
    raise RuntimeError("GPU not available. Please run change your runtime type and select GPU.")
DEVICE = torch.device("cuda")

# The SimCLR Method: Contrastive Learning

SimCLR learns representations by contrasting positive pairs against negative pairs in a latent space. Let's break down how this works:

## Contrastive Loss Function

Let $\text{sim}(u,v)$ denote the dot product between two normalized vectors $u$ and $v$ (i.e., cosine similarity).

The loss function for a positive pair of examples $(i,j)$ is defined as:

$$\ell_{i,j} = -\log \frac{\exp(\text{sim}(z_i, z_j)/\tau)}{\sum_{k=1}^{2N} \mathbf{1}_{[k \neq i]} \exp(\text{sim}(z_i, z_k)/\tau)}$$

Where:
- $\mathbf{1}_{[k \neq i]} \in \{0, 1\}$ is an indicator function evaluating to 1 if and only if $k \neq i$
- $\tau$ is a temperature parameter that controls the concentration level of the distribution
- $z_i$ and $z_j$ are the projected representations of two augmented views of the same image

## Key Components

1. **Positive pairs**: Different augmentations of the same image
2. **Negative pairs**: Augmentations of different images
3. **Temperature scaling**: Lower values of $\tau$ make the model more sensitive to hard negative examples

The final loss is computed by summing over all positive pairs and dividing by $2 \times N = \text{views} \times \text{batch\_size}$, where in SimCLR we typically have 2 views per image.

This approach encourages the model to pull representations of the same image closer together while pushing representations of different images apart in the embedding space.

**Note:**

# Implement the ContrastiveLoss in Pytorch

## Similarity Matrix Calculation

First, we need to implement the similarity matrix between all images in the batch by calculating the cosine similarity between all projections.

Complete the `calc_similarity_batch` function using PyTorch's [Cosine Similarity](https://pytorch.org/docs/stable/generated/torch.nn.functional.cosine_similarity.html) function.

**Important:** We need to compute the cosine similarity between all pairs of projections, resulting in an N×N matrix where N is the total number of representations in the batch (N=2*batch_size). Use [broadcasting semantics](https://pytorch.org/docs/stable/notes/broadcasting.html) to create this matrix efficiently.

In [ ]:
def calc_similarity_batch(a, b):
    #TODO: concat the representations
    representations = ...

    #TODO: calculate the cosine similarity
    similarity_matrix = ...

    return similarity_matrix

Let's run a dummy test to make sure that the output shape is correct. We'll create random embeddings and verify that our function returns a matrix with the expected dimensions.

In [ ]:
# Test calc_similarity_batch function
batch_size = 4  # Small batch size for testing
embedding_dim = 128  # Typical embedding dimension

# Create random embeddings
a = torch.randn(batch_size, embedding_dim)
b = torch.randn(batch_size, embedding_dim)

# Calculate similarity matrix
similarity_matrix = calc_similarity_batch(a, b)


# Verify the shape is (2*batch_size, 2*batch_size)
expected_shape = (2 * batch_size, 2 * batch_size)
assert similarity_matrix.shape == expected_shape, f"Similarity matrix has shape {similarity_matrix.shape} but expected {expected_shape}"
print("calc_similarity_batch function outputs the expected shape.")



# Defining the Contrastive Loss class

- First we need to normalize the output embeddings from our convolutional backbone. Normalizing embeddings ensures all representations lie on the unit hypersphere, preventing the model from artificially increasing distances between samples rather than learning meaningful representations.

- Next, we calculate the similarity matrix using the `calc_similarity_batch` function we implemented.

- Then we need to identify positive pairs within the similarity matrix. These positive pairs are shifted from the main diagonal by the batch size. We can access them using `torch.diag()`, which extracts the chosen diagonal from a matrix. We need to extract two diagonals: one at offset `batch_size` and another at offset `-batch_size`.

<p align="center"><br>
<img src="https://github.com/telecombcn-dl/labs-all/raw/main/labs/contrastive_learning/images/sim_matrix.png?raw=true" class="center" title="Sim_matrix" width="400"/>
</p><br>

- With all these components, we can calculate the contrastive loss function following the equation:

$$\ell_{i,j} = -\log \frac{\exp(\text{sim}(z_i, z_j)/\tau)}{\sum_{k=1}^{2N} \mathbf{1}_{[k \neq i]} \exp(\text{sim}(z_i, z_k)/\tau)}$$

Where:
- $\mathbf{1}_{[k \neq i]}$ is the mask that excludes the current sample from the denominator (implemented as `self.mask` in our class)
- $\tau$ is the temperature parameter that controls how concentrated the distribution becomes
- $z_i$ and $z_j$ are the projected representations of two augmented views of the same image


In [ ]:
class ContrastiveLoss(nn.Module):
    """
    Vanilla Contrastive loss, also called InfoNceLoss as in SimCLR paper
    """
    def __init__(self, batch_size, temperature=0.5):
        super().__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.mask = (~torch.eye(batch_size * 2, batch_size * 2, dtype=bool)).float()



    def forward(self, proj_1, proj_2):
        """
        proj_1 and proj_2 are batched embeddings [batch, embedding_dim]
        where corresponding indices are pairs
        z_i, z_j in the SimCLR paper
        """
        batch_size = proj_1.shape[0]
        #TODO: normalize the projections
        z_i = ...
        z_j = ...

        #TODO: calculate the similarity matrix
        similarity_matrix = ...

        #TODO: calculate the positive pairs
        sim_ij = ...
        sim_ji = ...

        positives = torch.cat([sim_ij, sim_ji], dim=0)

        #TODO: calculate the nominator
        nominator = ...

        #TODO: calculate the denominator (remember to move the mask to the same device as the similarity matrix)
        denominator = ...

        #TODO: calculate the loss
        all_losses = ...

        #Normalize the loss
        loss = torch.sum(all_losses) / (2 * self.batch_size)
        return loss

# Data augmentation

For the SimCLR pipeline, we need specific data augmentations to create different views of the same image. The Augment class below implements these key transformations:

1. **Random cropping**: Crops random portions of images at different scales
2. **Random color distortions**: Applies color jitter with randomized brightness, contrast, saturation, and hue
3. **Random Gaussian blur**: Applies blur with varying intensity
4. **Random grayscale conversion**: Occasionally converts images to grayscale

These augmentations are critical for SimCLR as they create semantically similar but visually different views of the same image. When applied to a single image, they produce two different views that maintain the same semantic content but with different visual appearances - these become our positive pairs.


In [ ]:
class Augment:
    """
    A stochastic data augmentation module
    Transforms any given data example randomly
    resulting in two correlated views of the same example,
    denoted x ̃i and x ̃j, which we consider as a positive pair.
    """

    def __init__(self, img_size, s=1):
        color_jitter = T.ColorJitter(
            0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s
        )

        self.train_transform = T.Compose(
            [
            T.RandomResizedCrop(size=img_size),
            T.RandomHorizontalFlip(p=0.5),  # with 0.5 probability
            T.RandomApply([color_jitter], p=0.8),
            T.RandomGrayscale(p=0.2),
            # imagenet stats
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]
        )

        self.test_transform = T.Compose(
            [
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        )

    def __call__(self, x):
        return self.train_transform(x), self.train_transform(x)

In [ ]:
import matplotlib.pyplot as plt

def imshow(img):
    """
    shows an imagenet-normalized image on the screen
    """
    mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
    std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
    unnormalize = T.Normalize((-mean / std).tolist(), (1.0 / std).tolist())
    npimg = unnormalize(img).numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

Visualize some of the images

In [ ]:
dataset = STL10("./", split='train', transform=Augment(96), download=True)
imshow(dataset[99][0][0])
imshow(dataset[99][0][0])
imshow(dataset[99][0][0])
imshow(dataset[99][0][0])

# Define the model

We will use the Resnet18 and we will modify the head to output embeddings instead of class probabilities.

In [ ]:
class SimCLR_ResNet(nn.Module):
    def __init__(self, pretrained=False, out_dim=128):
        super(SimCLR_ResNet, self).__init__()
        self.base_model = models.resnet18(pretrained=pretrained)
        #TODO: override the last layer of the base model
        self.base_model.fc = nn.Sequential(
            Linear layer of 512,512
            BatchNorm layer of 512
            ReLU activation
            Linear layer of 512,out_dim
            BatchNorm layer of out_dim

    def forward(self, x):
        x = self.base_model(x)
        return x

# SimCLR Training Process Explained

SimCLR performs self-supervised learning by creating two different augmented views of each image and training the network to recognize them as similar, while distinguishing them from other images.

<p align="center"><br>
<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNG90NGd1YWVlbGJwaHJiaTJoZ2M5MTM5czY3aWRueTNpYnRpZ2V6ZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/8SZhj0qY3XlHcr1150/giphy.gif" class="center" width="400"/>
</p><br>


SimCLR training involves several key steps:

1. **Views creation**: For each batch of images, we create two augmented views of each image.

2. **Forward Pass**: Both augmented views pass through the encoder network and projection head to extract embeddings.

3. **Contrastive Loss Calculation**: The embeddings are used to compute the contrastive loss:
    - Positive pairs (different views of same image) should have similar embeddings
    - Negative pairs (views from different images) should have dissimilar embeddings

4. **Optimization**: The network parameters are updated through backpropagation to minimize this contrastive loss.

5. **Downstream Evaluation**: After training, the encoder (without the projection head) can be used for downstream tasks like classification.

The temperature parameter τ controls the concentration level of the positive/negative sample distribution, affecting how hard the model considers negative examples.

In [ ]:
from tqdm.notebook import tqdm

def train_simclr(model, train_loader, optimizer, criterion, epochs, device, log_interval=10):
    """
    Train the SimCLR model

    Args:
        model: The SimCLR model
        train_loader: DataLoader for training data
        optimizer: Optimizer for training
        criterion: Contrastive loss function
        epochs: Number of epochs to train for
        device: Device to train on
        log_interval: How often to log progress

    Returns:
        List of training losses
    """
    model.train()
    losses = []

    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        epoch_pbar = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        for batch_idx, ((x_i, x_j), _) in enumerate(train_loader):
            # Move data to device
            x_i, x_j = x_i.to(device), x_j.to(device)

            # TODO: Forward pass through the model
            z_i = ...
            z_j = ...

            # TODO: Calculate contrastive loss
            loss = ...

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update running loss
            running_loss += loss.item()

            # Update progress bar
            epoch_pbar.set_postfix({"batch_loss": f"{loss.item():.6f}"})
            epoch_pbar.update(1)

            # Log progress to console
            if batch_idx % log_interval == 0 and batch_idx > 0:
                print(f'Epoch: {epoch+1}/{epochs}, Batch: {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.6f}')

        # Calculate average loss for epoch
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)

        # Close progress bar
        epoch_pbar.close()

        print(f'Epoch: {epoch+1}/{epochs}, Average Loss: {epoch_loss:.6f}')

    return losses

Let's train our SimCLR model for a single epoch on the unlabeled split of STL10 to verify our implementation works correctly.

In [ ]:
# Prepare the STL10 dataset with unlabeled data for SimCLR pretraining
batch_size = 256  # Larger batch size for contrastive learning
unlabeled_dataset = STL10(root="./", split='unlabeled', transform=Augment(96), download=True)
train_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=cpu_count())

# Define the model
model = SimCLR_ResNet(pretrained=False, out_dim=128).to(DEVICE)

# Define optimizer (Adam with weight decay as in SimCLR paper)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-6)

# Define contrastive loss
criterion = ContrastiveLoss(batch_size=batch_size, temperature=0.5).to(DEVICE)

# Training parameters
num_epochs = 1  # Reduced for demonstration, SimCLR typically uses 100+ epochs

# Train the model
loss_history = train_simclr(model, train_loader, optimizer, criterion, num_epochs, DEVICE, log_interval=100)

# Evaluate SimCLR performance

Training a SimCLR model from scratch requires significant computational resources and time. With standard Colab resources, we would reach the time limit before finishing the training.

To work around this limitation, we'll use a practical approach:

1. We've already trained a SimCLR model on the unlabeled split of STL10 for 300 epochs.
2. We'll load these pre-trained weights and evaluate their effectiveness on the labeled split of the STL10 dataset.
3. For comparison, we'll also evaluate:
    - A randomly initialized model (no pre-training)
    - An ImageNet pre-trained model

This approach allows us to understand the benefits of contrastive learning without requiring the extensive training time. The evaluation on the labeled split of STL10 will show us how well the representations learned through self-supervised contrastive learning transfer to supervised classification tasks compared to both random initialization and supervised pre-training on ImageNet.

First we will create the training loop for the supervised split

In [ ]:
import numpy as np

def train_evaluation(model, train_loader, test_loader, epochs=50, lr=0.001, device='cuda'):
    """
    Linear evaluation protocol: train a linear classifier on frozen features

    Args:
        model: The pretrained model without the projection head
        train_loader: DataLoader for labeled training data
        test_loader: DataLoader for labeled test data
        epochs: Number of epochs to train the linear classifier
        lr: Learning rate
        device: Device to train on

    Returns:
        Best test accuracy
    """

    # Create optimizer and loss function
    optimizer = torch.optim.SGD(model.base_model.parameters(), lr=lr)
    criterion = nn.NLLLoss()

    best_acc = 0.0

    for epoch in tqdm(range(epochs)):
        # Training
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        train_accuracy = 100.0 * correct / total

        # Evaluation
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

        test_accuracy = 100.0 * correct / total

        # Save best accuracy
        if test_accuracy > best_acc:
            best_acc = test_accuracy

        # Print progress
        if epoch % 5 == 0 or epoch == epochs - 1:
            print(f'Epoch: {epoch+1}/{epochs}, '
                  f'Train Loss: {train_loss/len(train_loader):.4f}, '
                  f'Train Acc: {train_accuracy:.2f}%, '
                  f'Test Loss: {test_loss/len(test_loader):.4f}, '
                  f'Test Acc: {test_accuracy:.2f}%')

    print(f'Best Test Accuracy: {best_acc:.2f}%')
    return best_acc

Now we will create an evaluation model for the supervised classification task on STL10. This model will be used to assess how well our self-supervised representation learning transfers to a downstream classification task.

In [ ]:
class SimCLR_ResNet_eval(nn.Module):
    def __init__(self, pretrained=False, out_dim=128):
        super(SimCLR_ResNet_eval, self).__init__()
        self.base_model = models.resnet18(pretrained=pretrained)
        #TODO: override the last layer of the base model
        self.base_model.fc = nn.Sequential(
            Linear layer of 512,out_dim
            LogSoftmax layer

    def forward(self, x):
        x = self.base_model(x)
        return x

Next, we will declare the labeled splits of STL10 and and we will train our model using the pretrained SimCLR weights.

In [ ]:
transform_preprocess = Augment(96).test_transform

# Create datasets
train_dataset = STL10(root="./", split='train', transform=transform_preprocess, download=True)
test_dataset = STL10(root="./", split='test', transform=transform_preprocess, download=True)

batch_size=128
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=cpu_count())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=cpu_count())


# Create a new model without projection head for evaluation (or modify existing model)
eval_model = SimCLR_ResNet_eval(pretrained=False, out_dim=10).to(DEVICE)

# Load weights from pretrained model (excluding projection head)
# Load the pretrained SimCLR weights
pretrained_weights = torch.load('./simclr_stl10_resnet18.pth', map_location=DEVICE)


# Remove the 'fc' layer from the state_dict
pretrained_weights_no_fc = {k: v for k, v in pretrained_weights["model_state_dict"].items() if not k.startswith('base_model.fc.')}

eval_model.load_state_dict(pretrained_weights_no_fc, strict=False)

# Evaluate with linear probing
simclr_pretrained = train_evaluation(eval_model, train_loader, test_loader, epochs=50, device=DEVICE)


In [ ]:
#TODO: Compare to a randomly initialized model for baseline
random_model = ...
random_accuracy = ...

In [ ]:
#TODO: Compare to a randomly initialized model for baseline
imagenet_model = ...
imagenet_model = ...

In [ ]:
# Visualize the comparison
import matplotlib.pyplot as plt

labels = ['Random Init', 'SimCLR Pretrained', 'Imagenet Pretrained']
accuracies = [random_accuracy, simclr_pretrained, imagenet_model]

plt.figure(figsize=(8, 6))
plt.bar(labels, accuracies, color=['red', 'green', 'blue'])
plt.ylabel('Test Accuracy (%)')
plt.title('Supervised Evaluation on STL-10')
plt.ylim(0, 100)
for i, v in enumerate(accuracies):
    plt.text(i, v + 1, f"{v:.2f}%", ha='center')
plt.show()

### Understanding Self-Supervised Learning Performance

Looking at our experimental results, we can observe the clear benefits of different pretraining strategies:

- **Random Initialization**: ~40% accuracy
- **SimCLR Pretraining**: ~75% accuracy 
- **ImageNet Pretraining**: ~86% accuracy

These results demonstrate the power of self-supervised learning. With SimCLR pretraining on unlabeled data, we achieved 76% accuracy, which is significantly better than training from scratch (40%) and approaches the performance of supervised pretraining on ImageNet (87%).

The key insight here is that SimCLR allows us to learn useful representations without requiring any labeled data. This is particularly valuable because:

1. **Reduced annotation costs**: Labeling large datasets like ImageNet requires thousands of human hours
2. **Domain adaptability**: Self-supervised methods can be applied to specialized domains where labeled data is scarce
3. **Efficiency**: We can leverage vast amounts of unlabeled data that would otherwise go unused

While ImageNet pretraining still performs better, the gap is remarkably small considering SimCLR doesn't use any labels. This demonstrates that contrastive learning effectively captures meaningful visual representations by learning to distinguish between different views of the same image versus views of different images.

For many real-world applications where labeled data is expensive or unavailable, SimCLR offers a powerful alternative that can deliver strong performance at a fraction of the annotation cost.

# Conclusions

## The Power of Self-Supervised Learning

In this lab, we've explored SimCLR, a powerful self-supervised learning framework that learns meaningful visual representations without requiring labeled data. Our experiments highlight several key insights:

- **Self-supervised learning is effective**: SimCLR achieved ~75% accuracy on STL10 classification without using any labels during pretraining, significantly outperforming random initialization (~40%).

- **Bridging the gap to supervised learning**: While ImageNet pretraining still leads (~86%), SimCLR approaches this performance without requiring expensive annotation, showing the efficiency of contrastive learning.

- **Practical implications**: For domains where labeled data is scarce or expensive to obtain, self-supervised approaches like SimCLR offer a compelling alternative that can leverage vast amounts of unlabeled data.

## Key Components of SimCLR's Success

The strong performance of SimCLR can be attributed to:

1. **Data augmentation strategy**: Creating diverse views of the same image helps the model learn robust features that are invariant to transformations.

2. **Contrastive loss function**: By pulling similar examples together and pushing dissimilar ones apart, the model learns discriminative features.

3. **Projection head architecture**: The non-linear projection head helps separate the representation space from the contrastive learning space.

SimCLR demonstrates that with the right learning framework, we can extract meaningful representations from unlabeled data that transfer well to downstream tasks, potentially reducing our dependence on large labeled datasets for visual recognition tasks.